In [1]:
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
import cv2
from pathlib import Path

In [9]:
paths = ["train/image", "test/image"] 
images = []  # Lista para almacenar las imágenes

for path in paths:
    imgs = Path(path).glob("**/*.png") 
    imgs = [str(x) for x in imgs]
    for img in imgs: 
        imagen = cv2.imread(img,1)
        imagen = cv2.resize(imagen, (160, 160), cv2.INTER_AREA)
        imagen = imagen / 255
        images.append(imagen)

images = np.array(images)
print(f"Mask images shape: {images.shape}")
    

Mask images shape: (100, 160, 160, 3)


In [10]:
paths = ["train/mask", "test/mask"] 
masks = []  # Lista para almacenar las imágenes

for path in paths:
    imgs = Path(path).glob("**/*.png") 
    imgs = [str(x) for x in imgs]
    for img in imgs: 
        imagen = cv2.imread(img,1)
        imagen = cv2.resize(imagen, (160, 160), cv2.INTER_AREA)
        imagen = imagen / 255
        masks.append(imagen)

masks = np.array(masks)
print(f"Images shape: {masks.shape}")
    

Images shape: (100, 160, 160, 3)


In [ ]:
def get_model(img_size=(160,160,3), num_clases = 1) :
    inputs = keras.Input(shape=img_size)
    #Primera mitas de la red: Extracción de las características

    #Bloque de entrada
    x =  keras.layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)

    #Parte residual
    previous_block = x

    #Bloques de convoluciEl bloque 1, 2 y 3 son iguales
    for filters in [64, 128, 256] : 
        x = keras.layers.Activation("relu")(x)
        x = keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation("relu")(x)
        x = keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Projección residual
        residual = keras.layers.Conv2D(filters, 1, strides=2, padding="same")(previous_block)
        x = keras.layers.add([x, residual])
        previous_block = keras